In [ ]:
import os

# Third-party
from astropy.constants import G
import astropy.coordinates as coord
from astropy.io import ascii, fits
import astropy.table as atbl
import astropy.units as u
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('notebook.mplstyle')
%matplotlib inline
import corner

import gala.dynamics as gd
import gala.coordinates as gc
import gala.potential as gp
from gala.units import galactic

In [ ]:
# Milky Way potential
mw_potential = gp.CCompositePotential()

mw_potential['disk'] = gp.MiyamotoNagaiPotential(m=6E10, a=3.5, b=0.14, units=galactic)
mw_potential['bulge'] = gp.HernquistPotential(m=1E10, c=1.1, units=galactic)

# for DM halo potential
M_h = 8E11 * u.Msun
rs_h = 20. * u.kpc
v_c = np.sqrt(((np.log(2.) - 0.5) * (G * M_h / rs_h)).decompose(galactic).value)
mw_potential['halo'] = gp.SphericalNFWPotential(v_c=v_c, r_s=rs_h, units=galactic)

In [ ]:
tgas_rave = fits.getdata('../data/tgas-rave.fits', 1)

SN_cut = 5

clean_idx = ((np.abs(tgas_rave['parallax'] / tgas_rave['parallax_error']) > SN_cut) & 
             np.isfinite(tgas_rave['parallax'] / tgas_rave['parallax_error']) &
             (tgas_rave['eHRV'] < 15.) &
             np.isfinite(tgas_rave['HRV'] / tgas_rave['eHRV']))

clean_tgas_rave = tgas_rave[clean_idx]
len(clean_tgas_rave)

In [ ]:
plt.hist(1/clean_tgas_rave['parallax'], bins=np.linspace(0,1,32));

In [ ]:
rv = clean_tgas_rave['HRV'] * u.km/u.s
c = coord.ICRS(ra=clean_tgas_rave['ra']*u.deg, dec=clean_tgas_rave['dec']*u.deg,
               distance=(clean_tgas_rave['parallax']*u.mas).to(u.pc, equivalencies=u.parallax()))
pm = np.vstack((clean_tgas_rave['pmra'],clean_tgas_rave['pmdec'])) * u.mas/u.yr

xyz = c.transform_to(coord.Galactocentric).cartesian.xyz
vxyz = gc.vhel_to_gal(c, pm=pm, rv=rv)

# _ix = np.abs(xyz[2]) < 100*u.pc
w0 = gd.CartesianPhaseSpacePosition(pos=xyz, vel=vxyz)

In [ ]:
w0.shape

In [ ]:
all_ecc = np.zeros(w0.shape[0])
z_max = np.zeros(w0.shape[0])
for i in range(w0.shape[0]):
    orbit = mw_potential.integrate_orbit(w0[i], dt=-1., n_steps=1000)
    all_ecc[i] = orbit.eccentricity()
    z_max[i] = np.abs(orbit.pos[2]).max().to(u.kpc).value

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(all_ecc, clean_tgas_rave['Fe_H'], c=z_max,
            marker='.', alpha=0.5, cmap='plasma_r', vmin=0, vmax=1)
plt.xlim(-0.025, 1)
plt.ylim(-1.2, 0.5)
plt.xlabel('$e$')
plt.ylabel('[Fe/H]')
cb = plt.colorbar()
cb.set_label(r'$z_{\rm max}$')

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(all_ecc, z_max, c=clean_tgas_rave['Fe_H'],
            marker='.', alpha=0.5, cmap='plasma', vmin=-1., vmax=0)
plt.xlim(-0.025, 1)
plt.ylim(0,2)
plt.xlabel(r'$e$')
plt.ylabel(r'$z_{\rm max}$')

In [ ]:
plt.hist(z_max, bins=np.linspace(0,10.,32));
plt.yscale('log')
plt.xlabel(r'$z_{\rm max}$')

In [ ]:
for_keith = clean_tgas_rave[((z_max > 3.) & np.isfinite(z_max) & (clean_tgas_rave['Fe_H'] > -0.5))]
len(for_keith)

In [ ]:
fits.BinTableHDU(for_keith).writeto('../data/high-zmax-for-keith.fits')

In [ ]:
for_keith.dtype.names